In [1]:
# Install necessary libraries
# ----------------------------------------
!pip install ultralytics torch torchvision torchaudio gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Clone the BCCD Dataset from GitHub
# ----------------------------------------
!git clone https://github.com/Shenggan/BCCD_Dataset.git

Cloning into 'BCCD_Dataset'...
remote: Enumerating objects: 800, done.
remote: Total 800 (delta 0), reused 0 (delta 0), pack-reused 800 (from 1)
Receiving objects: 100% (800/800), 7.39 MiB | 11.09 MiB/s, done.
Resolving deltas: 100% (378/378), done.


In [3]:
# Organize the BCCD Dataset
# ----------------------------------------
import os
import random
import shutil

In [4]:
# Paths to images and annotations
dataset_path = 'BCCD_Dataset/BCCD/JPEGImages'
label_path = 'BCCD_Dataset/BCCD/Annotations'

In [5]:

# List of image filenames
images = [f for f in os.listdir(dataset_path) if f.endswith('.jpg')]
random.shuffle(images)

In [6]:
# Create train/val directories
os.makedirs('train/images', exist_ok=True)
os.makedirs('train/labels', exist_ok=True)
os.makedirs('val/images', exist_ok=True)
os.makedirs('val/labels', exist_ok=True)

In [7]:
# Split into train (80%) and val (20%)
split_idx = int(0.8 * len(images))
train_images = images[:split_idx]
val_images = images[split_idx:]



In [8]:
# Move train images and labels
for img in train_images:
    shutil.move(os.path.join(dataset_path, img), 'train/images')
    shutil.move(os.path.join(label_path, img.replace('.jpg', '.xml')), 'train/labels')


In [9]:
# Move val images and labels
for img in val_images:
    shutil.move(os.path.join(dataset_path, img), 'val/images')
    shutil.move(os.path.join(label_path, img.replace('.jpg', '.xml')), 'val/labels')

print("Dataset split into train and val folders successfully.")

Dataset split into train and val folders successfully.


In [10]:
# Create the data.yaml file for YOLOv8
# ----------------------------------------
data_yaml = """
train: ./train/images
val: ./val/images
nc: 3
names: ['RBC', 'WBC', 'Platelets']
"""
with open('data.yaml', 'w') as f:
    f.write(data_yaml)
print("data.yaml created!")

data.yaml created!


In [12]:
# Fine-tune YOLOv8 on the BCCD dataset with data augmentation
# ----------------------------------------
from ultralytics import YOLO

# Load the pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')  # yolov8n.pt is the lightweight version

# Train the model with data augmentation for 50 epochs
# Add `val=False` to skip validation during training and metrics calculation
model.train(data='/content/data.yaml', epochs=50, imgsz=640, augment=True, val=False)

# Save the fine-tuned model
model.save('best_model.pt')
print("Model saved successfully!")


Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=False, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

train: Scanning /content/train/labels.cache... 0 images, 291 backgrounds, 0 corrupt: 100%|██████████| 291/291 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/train/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/val/labels.cache... 0 images, 73 backgrounds, 0 corrupt: 100%|██████████| 73/73 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/val/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      49/50      2.58G          0      2.153          0          0        640: 100%|██████████| 19/19 [00:03<00:00,  5.57it/s]



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      2.58G          0      2.135          0          0        640: 100%|██████████| 19/19 [00:04<00:00,  4.06it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.95it/s]


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [18]:
# Define a function to perform model inference on an image
# ----------------------------------------
import gradio as gr

# Function to perform inference and return results with bounding boxes and cell type
def predict(image):
    results = model(image)  # Perform inference on the input image
    labels = results.names  # Get the class names from the model (RBC, WBC, Platelets)
    detections = results[0].boxes  # Get bounding boxes and other detection details

    # Extract detected cells, bounding boxes, and confidence scores
    detection_info = []
    for det in detections:
        class_id = int(det.cls)  # Class ID (0 for RBC, 1 for WBC, 2 for Platelets)
        cell_type = labels[class_id]  # Get the label (RBC, WBC, Platelets)
        confidence = det.conf  # Confidence score
        bbox = det.xyxy  # Bounding box coordinates (x1, y1, x2, y2)
        detection_info.append(f"Detected {cell_type} with confidence {confidence:.2f}")

    # Show the image with bounding boxes
    image_with_boxes = results[0].plot()  # This shows bounding boxes on the image

    # Join the detection information into a string to display in the output
    detection_text = "\n".join(detection_info)

    return image_with_boxes, detection_text  # Return the image with boxes and the detection info


In [19]:
# Create Gradio interface
# ----------------------------------------
iface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Image(type="pil"), gr.Textbox()],
    title="Blood Cell Detection",
    description="Upload an image to detect RBCs, WBCs, and Platelets with confidence scores."
)

# Launch the Gradio app
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://83a6a45f282ef61c38.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
